## Call function

In [49]:
def flatten_dict(dd, separator='_', prefix=''):
    return {f"{prefix}{separator}{k}" if prefix else k: v
            for kk, vv in dd.items()
            for k, v in (flatten_dict(vv, separator, kk).items() if isinstance(vv, dict) else {kk: vv}.items())}

In [50]:
import requests

def call_api(year,month,day):
    

    url = "https://partners.api.skyscanner.net/apiservices/v3/flights/indicative/search"

    payload = { "query": {
            "market": "UK",
            "locale": "en-GB",
            "currency": "EUR",
            "queryLegs": [
                {
                    "originPlace": { "queryPlace": { "iata": orig } },
                    "destinationPlace": { "queryPlace": { "iata": dest } },
                    "fixedDate": {
                        "year": year,
                        "month": month,
                        "day": day
                    }
                }
            ]
        } }
    headers = {
        "x-api-key": "prtl6749387986743898559646983194"
    }

    response = requests.post(url, json=payload, headers=headers)
    output=response.json()
    results=output["content"]["results"]
    import pandas as pd
    import json

    quotes=results["quotes"]
    quotes_df=pd.json_normalize(quotes)
    quotes_df.columns=[col.split(".")[-1] for col in quotes_df.columns]

    flattened_results=[flatten_dict(result) for result in quotes.values()]
    
    quote_df = pd.DataFrame.from_records(flattened_results)
    return quote_df

----

In [51]:
orig = input("Origin IATA: ")
dest = input("Destiny IATA: ")
date_from=input("Search date from (dd/mm/yyyy): ")
date_to=input("Search date to (dd/mm/yyyy): ")

Origin IATA: LIS
Destiny IATA: LTN
Search date from (dd/mm/yyyy): 20/05/2023
Search date to (dd/mm/yyyy): 25/05/2023


In [52]:
from datetime import datetime
import pandas as pd

date_from = pd.to_datetime(date_from, format='%d/%m/%Y')
date_to = pd.to_datetime(date_to, format='%d/%m/%Y')

dates = pd.date_range(date_from, date_to, freq='D')

In [53]:
dates

DatetimeIndex(['2023-05-20', '2023-05-21', '2023-05-22', '2023-05-23',
               '2023-05-24', '2023-05-25'],
              dtype='datetime64[ns]', freq='D')

In [54]:
dates[0].year

2023

In [55]:
dates[0].month

5

In [56]:
len(dates)

6

-----

In [57]:
quote_df_list=[]
    
for i in range(0,len(dates)):
    orig=orig
    dest=dest
    year=dates[i].year
    month=dates[i].month
    day=dates[i].day
    

    quote_df_list.append(call_api(year,month,day))

In [23]:
quote_df_list[1]

,minPrice_amount,minPrice_unit,minPrice_updateStatus,isDirect,outboundLeg_originPlaceId,outboundLeg_destinationPlaceId,outboundLeg_departureDateTime_year,outboundLeg_departureDateTime_month,outboundLeg_departureDateTime_day,outboundLeg_departureDateTime_hour,...,inboundLeg_originPlaceId,inboundLeg_destinationPlaceId,inboundLeg_departureDateTime_year,inboundLeg_departureDateTime_month,inboundLeg_departureDateTime_day,inboundLeg_departureDateTime_hour,inboundLeg_departureDateTime_minute,inboundLeg_departureDateTime_second,inboundLeg_quoteCreationTimestamp,inboundLeg_marketingCarrierId
0,138,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,21,0,...,,,0,0,0,0,0,0,0,
1,123,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,95565053,2023,5,21,0,...,,,0,0,0,0,0,0,0,


In [24]:
for i in range (0,len(quote_df_list)):
    if len(quote_df_list[i]) > 1:
        quote_df_list[i]['minPrice_amount'] = pd.to_numeric(quote_df_list[i]['minPrice_amount'])
        min_idx = quote_df_list[i]['minPrice_amount'].idxmin()
        quote_df_list[i]=quote_df_list[i].loc[[min_idx]]

In [25]:
quote_df_list[1]

,minPrice_amount,minPrice_unit,minPrice_updateStatus,isDirect,outboundLeg_originPlaceId,outboundLeg_destinationPlaceId,outboundLeg_departureDateTime_year,outboundLeg_departureDateTime_month,outboundLeg_departureDateTime_day,outboundLeg_departureDateTime_hour,...,inboundLeg_originPlaceId,inboundLeg_destinationPlaceId,inboundLeg_departureDateTime_year,inboundLeg_departureDateTime_month,inboundLeg_departureDateTime_day,inboundLeg_departureDateTime_hour,inboundLeg_departureDateTime_minute,inboundLeg_departureDateTime_second,inboundLeg_quoteCreationTimestamp,inboundLeg_marketingCarrierId
1,123,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,95565053,2023,5,21,0,...,,,0,0,0,0,0,0,0,


In [26]:
df = pd.concat(quote_df_list, ignore_index=True)
df

,minPrice_amount,minPrice_unit,minPrice_updateStatus,isDirect,outboundLeg_originPlaceId,outboundLeg_destinationPlaceId,outboundLeg_departureDateTime_year,outboundLeg_departureDateTime_month,outboundLeg_departureDateTime_day,outboundLeg_departureDateTime_hour,...,inboundLeg_originPlaceId,inboundLeg_destinationPlaceId,inboundLeg_departureDateTime_year,inboundLeg_departureDateTime_month,inboundLeg_departureDateTime_day,inboundLeg_departureDateTime_hour,inboundLeg_departureDateTime_minute,inboundLeg_departureDateTime_second,inboundLeg_quoteCreationTimestamp,inboundLeg_marketingCarrierId
0,99,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,20,0,...,,,0,0,0,0,0,0,0,
1,123,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,95565053,2023,5,21,0,...,,,0,0,0,0,0,0,0,
2,99,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,22,0,...,,,0,0,0,0,0,0,0,
3,55,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,23,0,...,,,0,0,0,0,0,0,0,
4,40,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,24,0,...,,,0,0,0,0,0,0,0,


In [28]:
dates_df = pd.DataFrame({'Date': dates})

In [29]:
result = pd.concat([dates_df, df], axis=1)
result

,Date,minPrice_amount,minPrice_unit,minPrice_updateStatus,isDirect,outboundLeg_originPlaceId,outboundLeg_destinationPlaceId,outboundLeg_departureDateTime_year,outboundLeg_departureDateTime_month,outboundLeg_departureDateTime_day,...,inboundLeg_originPlaceId,inboundLeg_destinationPlaceId,inboundLeg_departureDateTime_year,inboundLeg_departureDateTime_month,inboundLeg_departureDateTime_day,inboundLeg_departureDateTime_hour,inboundLeg_departureDateTime_minute,inboundLeg_departureDateTime_second,inboundLeg_quoteCreationTimestamp,inboundLeg_marketingCarrierId
0,2023-05-20,99,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,20,...,,,0,0,0,0,0,0,0,
1,2023-05-21,123,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,95565053,2023,5,21,...,,,0,0,0,0,0,0,0,
2,2023-05-22,99,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,22,...,,,0,0,0,0,0,0,0,
3,2023-05-23,55,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,23,...,,,0,0,0,0,0,0,0,
4,2023-05-24,40,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,24,...,,,0,0,0,0,0,0,0,


----

----

## Clean version

##### Functions

In [58]:
def flatten_dict(dd, separator='_', prefix=''):
    return {f"{prefix}{separator}{k}" if prefix else k: v
            for kk, vv in dd.items()
            for k, v in (flatten_dict(vv, separator, kk).items() if isinstance(vv, dict) else {kk: vv}.items())}

In [59]:
import requests

def call_api(year,month,day):
    

    url = "https://partners.api.skyscanner.net/apiservices/v3/flights/indicative/search"

    payload = { "query": {
            "market": "UK",
            "locale": "en-GB",
            "currency": "EUR",
            "queryLegs": [
                {
                    "originPlace": { "queryPlace": { "iata": orig } },
                    "destinationPlace": { "queryPlace": { "iata": dest } },
                    "fixedDate": {
                        "year": year,
                        "month": month,
                        "day": day
                    }
                }
            ]
        } }
    headers = {
        "x-api-key": "prtl6749387986743898559646983194"
    }

    response = requests.post(url, json=payload, headers=headers)
    output=response.json()
    results=output["content"]["results"]
    import pandas as pd
    import json

    quotes=results["quotes"]
    quotes_df=pd.json_normalize(quotes)
    quotes_df.columns=[col.split(".")[-1] for col in quotes_df.columns]

    flattened_results=[flatten_dict(result) for result in quotes.values()]
    
    quote_df = pd.DataFrame.from_records(flattened_results)
    return quote_df

In [60]:
def dates(date_from,date_to):
    from datetime import datetime
    import pandas as pd

    date_from = pd.to_datetime(date_from, format='%d/%m/%Y')
    date_to = pd.to_datetime(date_to, format='%d/%m/%Y')

    dates = pd.date_range(date_from, date_to, freq='D')
    return dates

In [61]:
def get_list(dates,orig,dest):
    quote_df_list=[]

    for i in range(0,len(dates)):
        orig=orig
        dest=dest
        year=dates[i].year
        month=dates[i].month
        day=dates[i].day

        quote_df_list.append(call_api(year,month,day))
        
    for i in range (0,len(quote_df_list)):
        if len(quote_df_list[i]) > 1:
            quote_df_list[i]['minPrice_amount'] = pd.to_numeric(quote_df_list[i]['minPrice_amount'])
            min_idx = quote_df_list[i]['minPrice_amount'].idxmin()
            quote_df_list[i]=quote_df_list[i].loc[[min_idx]]
            
    df = pd.concat(quote_df_list, ignore_index=True)
    dates_df = pd.DataFrame({'Date': dates})
    result = pd.concat([dates_df, df], axis=1)
    
    return result

##### User interaction

In [62]:
orig = input("Origin IATA: ")
dest = input("Destiny IATA: ")
date_from=input("Search date from (dd/mm/yyyy): ")
date_to=input("Search date to (dd/mm/yyyy): ")

get_list(dates(date_from,date_to),orig,dest)

Origin IATA: LIS
Destiny IATA: LTN
Search date from (dd/mm/yyyy): 20/05/2023
Search date to (dd/mm/yyyy): 25/05/2023


,Date,minPrice_amount,minPrice_unit,minPrice_updateStatus,isDirect,outboundLeg_originPlaceId,outboundLeg_destinationPlaceId,outboundLeg_departureDateTime_year,outboundLeg_departureDateTime_month,outboundLeg_departureDateTime_day,...,inboundLeg_originPlaceId,inboundLeg_destinationPlaceId,inboundLeg_departureDateTime_year,inboundLeg_departureDateTime_month,inboundLeg_departureDateTime_day,inboundLeg_departureDateTime_hour,inboundLeg_departureDateTime_minute,inboundLeg_departureDateTime_second,inboundLeg_quoteCreationTimestamp,inboundLeg_marketingCarrierId
0,2023-05-20,99,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,20,...,,,0,0,0,0,0,0,0,
1,2023-05-21,123,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,False,95565055,95565053,2023,5,21,...,,,0,0,0,0,0,0,0,
2,2023-05-22,99,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,22,...,,,0,0,0,0,0,0,0,
3,2023-05-23,55,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,23,...,,,0,0,0,0,0,0,0,
4,2023-05-24,40,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,24,...,,,0,0,0,0,0,0,0,
5,2023-05-25,44,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,25,...,,,0,0,0,0,0,0,0,
